# Hyperexponential Static Schedule

In [38]:
import numpy as np
import scipy, math
from scipy.stats import binom, erlang, poisson
from scipy.optimize import minimize

In [40]:
def SCV_to_params(SCV):
    
    # weighted Erlang case
    if SCV <= 1:
        K = math.floor(1/SCV)
        p = ((K + 1) * SCV - math.sqrt((K + 1) * (1 - K * SCV))) / (SCV + 1)
        mu = K + (1 - p)
    
        return K, p, mu
    
    # hyperexponential case
    else:
        p = 0.5 * (1 + np.sqrt((SCV - 1) / (SCV + 1)))
        mu = 1 # 1 / mean
        mu1 = 2 * p * mu
        mu2 = 2 * (1 - p) * mu
        
        return p, mu1, mu2

In [61]:
n = 3
omega = 0.5
SCV = 1.3
p, mu1, mu2 = SCV_to_params(SCV)

In [62]:
def trans_p(k,l,y,z,t,p,mu1,mu2):
    
    # 1. No client has been served before time t.
    if l == k+1 and z == y:
        if y == 1:
            return np.exp(-mu1 * t)
        elif y == 2:
            return np.exp(-mu2 * t)
    
    # 2. All clients have been served before time t.
    elif l == 1:
        if y == 1:
            prob = sum([binom.pmf(m, k-1, p) * psi(t, m+1, k-1-m, mu1, mu2) for m in range(k)])
            if z == 1:
                return p * prob
            elif z == 2:
                return (1-p) * prob
        elif y == 2:
            prob = sum([binom.pmf(m, k-1, p) * psi(t, m, k-m, mu1, mu2) for m in range(k)])
            if z == 1:
                return p * prob
            elif z == 2:
                return (1-p) * prob
    
    # 3. Some (but not all) clients have been served before time t.
    elif 2 <= l <= k:
        if y == 1:
            prob_diff = sum([binom.pmf(m, k-l, p) * psi(t, m+1, k-l-m, mu1, mu2) for m in range(k-l+1)]) \
                            - sum([binom.pmf(m, k-l+1, p) * psi(t, m+1, k-l+1-m, mu1, mu2) for m in range(k-l+2)])
            if z == 1:
                return p * prob_diff
            elif z == 2:
                return (1-p) * prob_diff
        elif y == 2:
            prob_diff = sum([binom.pmf(m, k-l, p) * psi(t, m, k-l+1-m, mu1, mu2) for m in range(k-l+1)]) \
                            - sum([binom.pmf(m, k-l+1, p) * psi(t, m, k-l+2-m, mu1, mu2) for m in range(k-l+2)])
            if z == 1:
                return p * prob_diff
            elif z == 2:
                return (1-p) * prob_diff
    
    # any other case is invalid
    return 0


In [63]:
def zeta(alpha, t, k):
    
    if not k:
        return (np.exp(alpha * t) - 1) / alpha
    else:
        return ((t ** k) * np.exp(alpha * t) - k * zeta(alpha, t, k-1)) / alpha

def rho(t,m,k,mu1,mu2):
    
    if not k:
        return np.exp(-mu2 * t) * (mu1 ** m) / ((mu1 - mu2) ** (m + 1)) * erlang.cdf(t, m+1, scale=1/(mu1 - mu2))
    elif not m:
        return np.exp(-mu1 * t) * (mu2 ** k) / math.factorial(k) * zeta(mu1-mu2, t, k)
    else:
        return (mu1 * rho(t, m-1, k, mu1, mu2) - mu2 * rho(t, m, k-1, mu1, mu2)) / (mu1 - mu2)

def psi(t,m,k,mu1,mu2):
    
    if not m:
        return erlang.cdf(t, k, scale=1/mu2)
    else:
        return erlang.cdf(t, m, scale=1/mu1) - mu1 * sum([rho(t, m-1, i, mu1, mu2) for i in range(k)])
    
def sigma(t,m,k,mu1,mu2):
    
    if not k:
        return t * erlang.cdf(t, m, scale=1/mu1) - (m / mu1) * erlang.cdf(t, m+1, scale=1/mu1)
    elif not m:
        return t * erlang.cdf(t, k, scale=1/mu2) - (k / mu2) * erlang.cdf(t, k+1, scale=1/mu2)
    else:
        return (t - k / mu2) * erlang.cdf(t, m, scale=1/mu1) - (m / mu1) * erlang.cdf(t, m+1, scale=1/mu1) \
                    + (mu1 / mu2) * sum([(k - i) * rho(t, m-1, i, mu1, mu2) for i in range(k)])


In [64]:
def f_bar(t,k,y,p,mu1,mu2):
    
    if y == 1:
        return sum([binom.pmf(m, k-1, p) * sigma(t, m+1, k-1-m, mu1, mu2) for m in range(k)])
    elif y == 2:
        return sum([binom.pmf(m, k-1, p) * sigma(t, m, k-m, mu1, mu2) for m in range(k)])

def h_bar(k,y,mu1,mu2):
    
    if k == 1:
        return 0
    else:
        if y == 1:
            return (k-2) + (1/mu1)
        elif y == 2:
            return (k-2) + (1/mu2)

def compute_probs_hyp(t,p,mu1,mu2):
    """
    Computes P(N_ti = j, Z_ti = z) for i=1,...,n, j=1,...,i and z=1,2.
    """
    
    n = len(t)
    probs = [[[None for z in range(2)] for j in range(i+1)] for i in range(n)]
    
    probs[0][0][0] = p
    probs[0][0][1] = 1 - p
    
    for i in range(2,n+1):
        
        x_i = t[i-1] - t[i-2]
        
        for j in range(1,i+1):
            for z in range(1,3):
                probs[i-1][j-1][z-1] = 0

                for k in range(max(1,j-1),i):
                    for y in range(1,3):
                        probs[i-1][j-1][z-1] += trans_p(k,j,y,z,x_i,p,mu1,mu2) * probs[i-2][k-1][y-1]
    return probs

def static_cost_hyp(t,p,mu1,mu2,omega):
    """
    Computes the cost of a static schedule in the weighted Erlang case.
    """
    
    n = len(t)
    
    # total expected waiting/idle time
    sum_EW, sum_EI = 0, 0
    probs = compute_probs_hyp(t, p, mu1, mu2)
    
    for i in range(2,n+1):
        
        # waiting time
        for k in range(2,i+1):
            sum_EW += h_bar(k, 1, mu1, mu2) * probs[i-1][k-1][0] + h_bar(k, 2, mu1, mu2) * probs[i-1][k-1][1]
        
        # idle time
        for k in range(1,i):                
            x_i = t[i-1] - t[i-2]
            sum_EI += f_bar(x_i, k, 1, p, mu1, mu2) * probs[i-2][k-1][0] + f_bar(x_i, k, 2, p, mu1, mu2) * probs[i-2][k-1][1]
    
    return omega * sum_EI + (1 - omega) * sum_EW


In [65]:
static_cost_hyp(range(n),p,mu1,mu2,omega)

0.8793292712926135

In [69]:
optimization = minimize(static_cost_hyp, range(n), args=(p,mu1,mu2,omega))
sign = 1
if optimization.x[0] < 0:
    sign = -1

optimization.x += optimization.x[0] * -sign # let the schedule start at time 0
print(optimization)

      fun: 0.8713678479409223
 hess_inv: array([[ 1.29256506,  0.0621651 , -0.35473126],
       [ 0.0621651 ,  1.18482708, -0.24699555],
       [-0.35473126, -0.24699555,  1.60173126]])
      jac: array([-7.45058060e-08, -2.23517418e-08,  8.94069672e-08])
  message: 'Optimization terminated successfully.'
     nfev: 30
      nit: 5
     njev: 6
   status: 0
  success: True
        x: array([0.        , 0.8115546 , 1.82727047])


## Web Scraping

In [70]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import pandas as pd

In [71]:
url = f'http://www.appointmentscheduling.info/index.php?SCV={SCV}&N={n}&omega={omega}&objFun=1'

# opening up connection, grabbing the page
uClient = urlopen(url)
page_html = uClient.read()
uClient.close()

# html parsing
page_soup = soup(page_html, "html.parser")
table = page_soup.findAll("table", {"class": "bordered"})[1]

# get appointment schedule
df = pd.read_html(str(table))[0]
schedule = df[df.columns[2]].values[:-2]

In [72]:
schedule

array([0.    , 0.829 , 1.8235])

In [73]:
static_cost_hyp(schedule,p,mu1,mu2,omega)

0.8714665901722874